# General Code

## Startup Code

In [19]:
import random
from enum import Enum
import math
import webbrowser
import ipywidgets as widgets
import urllib
import pyfiglet


In [20]:
class Patterns(Enum):
    major = 1
    natural_minor = 2


In [21]:
notes      = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
note_names = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']
keys       = [['C',0], ['G',1], ['D',2], ['A',3], ['E',4], ['B',5], ['F#/Gb',6], ['Db',-5], ['Ab',-4], ['Eb',-3], ['Bb', -2], ['F',-1]]
intervals  = ['P1','m2','M2','m3','M3','P4','a4/d5','P5','m6','M6','m7','M7']
modes      = ['Ionian', 'Dorian', 'Phrygian', 'Lydian', 'Mixolydian', 'Aeolian', 'Locrian']

major_scale         = [  1,  2,  3,  4,  5,  6,  7,  8 ]
natural_minor_scale = [  1,  2, -3,  4,  5, -6, -7,  8 ]
major_intervals     = [  0,  2,  2,  1,  2,  2,  2,  1 ]

practices = [
    'scales/intervals on guitar',
    'scales/intervals on keyboard',
]


In [22]:
# Return the simple offset from the tone name string entry and integer offset (i.e., ['cb',-3] ==> 'G' )
def relativeToneLetter(tone, offset):
    width = ord('G') - ord('A') + 1
    i = ord(tone[0].upper()) + offset
    if (i < ord('A')): i += width
    if (i > ord('G')): i -= width
    return chr(i)


# Return the full tone name from a string entry (i.e., 'bb' ==> 'A#/Bb')
def fullToneName(tone):
    if len(tone) < 2: return tone.upper()
    tone_name = tone[0].upper() + tone[-1:]
    if (tone_name[1] == 'b'):
        tone_name = relativeToneLetter(tone_name[0],-1) + '#/' + tone_name
    elif (tone_name[1] == '#'):
        tone_name = tone_name + '/' + relativeToneLetter(tone_name[0],1) + 'b'
    try:
        i = notes.index(tone_name)
        return tone_name
    except ValueError:
        return 'X'


In [23]:
# Take a scale as an array of strings, and select any split note names
# like "C#/Db" to create a well-ordered scale
def adjustScale(scale):
    root = scale[0]
    if root == 'F' or (len(root) > 1 and root[1] == 'b'):
        flat = True
    else:
        flat = False
    letter = root[0]
    new_scale = []
    for i, tone in enumerate(scale):
        if i == 0:
            new_scale.append(root)
        else:
            note = tone
            if tone[0] != relativeToneLetter(letter,1):
                if len(tone) == 1:
                    if tone[0] == letter:
                        note = relativeToneLetter(letter,1) + 'b'
                    else:
                        note = relativeToneLetter(letter,1) + '#'
                else:
                    note = tone.split('/')[1]
            elif len(tone) > 2:
                note = tone.split('/')[1 if flat else 0]
            letter = note[0]
            new_scale.append(note)
    return new_scale


## Basic Functions

In [24]:
# Return a scale as an array of strings for a given 'root' note string
# like "Ab", and a 'pattern' designation like Pattern.major. The major
# diatonic scale will be returned if the 'pattern' argument is left out.
def getScale(root, pattern=Patterns.major):
    scale = []
    tone = notes.index(fullToneName(root))

    scale_pattern = major_scale
    if (pattern == Patterns.natural_minor):
        scale_pattern = natural_minor_scale

    for degree in scale_pattern:
        tone = (tone + major_intervals[abs(degree)-1]) % len(notes)
        scale.append(notes[ (tone-1) if degree < 0 else tone ])

    scale[0] = root.upper()
    if len(scale[0]) > 1 and scale[0][1] == 'B':
        scale[0] = scale[0][0] + 'b'

    return adjustScale(scale)


# Returns frequency of a note in Hz (i.e., the freq of middle C would be freq('C',4) ==> 261.625...)
def noteToFreq(note, octave):
    x = 12*(octave-4)+notes.index(note)-9
    return 440.*(2**(1./12.))**x


# Returns the closest note letter, octave, and deviation from center (in cent) to a given frequency (i.e., note(261.3) ==> ('C', 4, -2.1556...))
def freqToNote(freq_in):
    x = math.log(freq_in / 440., 2**(1./12.))
    octave = int(4 + (x + 9)/12)
    note_a = int((x + 9) % 12)

    f = noteToFreq(notes[note_a],octave)
    cent = 1200.*math.log2(freq_in/f)
    if abs(cent) > 50:
        note_a = (note_a + 1) if note_a < 11 else 0
        octave += 1 if note_a == 0 else 0
        f = noteToFreq(notes[note_a],octave)
        cent = 1200.*math.log2(freq_in/f)

    return(notes[note_a], octave, cent)

# Returns the index into the notes[] array for the given note name (i.e., note_index('bb') ==> 10)
def noteIndex(note):
    if len(note) == 1:
        return notes.index(note.upper())
    else:
        a = note[0].upper() + note[1]
        for i in range(len(notes)):
            if notes[i].find(a) != -1:
                return i
    return -1

# Returns the musical interval between the given root and target notes (i.e., interval('bb','g#') ==> 'm7')
def getInterval(root, note):
    ir = noteIndex(root)
    iv = noteIndex(note)
    d = ((iv - ir) + (12 if ((iv - ir) < 0) else 0)) % 12
    return intervals[d]

### Get a Random Key and its Major Scale

In [25]:
key = random.randint(0, len(keys)-1)
print(keys[key][0])

getScale(keys[key][0])


A


['A', 'B', 'C#', 'D', 'E', 'F#', 'G#', 'A']

# Reference

<pre>
Major scale: 1 2 3 4 5 6 7 8'
  intervals:  W W H W W W H

Natural minor scale: 1 2 <sup>b</sup>3 4 5 6 7 8'
          intervals:  W H  W W H W W
</pre>

A *natural* minor scale can be built from the same set of notes as a major scale. The key of the natural minor is the 6<sup>th</sup> degree of the major scale.

### Pattern System

<pre>
Patterns along a string:
  X = 0 2 4
  Y = 0 1 3
  Z = 0 2 3

Patterns accross strings (root is on first note):
  1 Ionian     XX (YYZZ) = Major
  2 Dorian     ZX (XXYY)
  3 Phrygian   YZ (ZXXX)
  4 Lydian     XY (YZZX)
  5 Mixolydian XXX (YYZ)
  6 Aeolian    ZZ (XXXY) = Natural Minor
  7 Locrian    YY (ZZXX)

Rules across strings:
  1. Move up a fret from X to Y pattern
  2. Move up a fret from G string to B string

Rules along strings:
  1. Start next pattern on next fret from pattern X to X
  2. Start next pattern on 2nd fret after any other patterns
</pre>

# What Notes are in the Harmonic Series?

In [26]:
C4 = noteToFreq('C',4)

print('Harmonic series of middle C:')

print(f' Harm   Freq[Hz]    Note   Err    Int')
for i in range(15):
    h = i+1
    f = C4*h
    n,o,c = freqToNote(f)
    iv = getInterval('C',n)
    print(f'  {h:2d}    {f:7.2f}   {n:>5s}{o}   {c:+3.0f}   {iv:>5s}')

print('...')


Harmonic series of middle C:
 Harm   Freq[Hz]    Note   Err    Int
   1     261.63       C4    +0      P1
   2     523.25       C5    -0      P1
   3     784.88       G5    +2      P5
   4    1046.50       C6    -0      P1
   5    1308.13       E6   -14      M3
   6    1569.75       G6    +2      P5
   7    1831.38   A#/Bb6   -31      m7
   8    2093.00       C7    -0      P1
   9    2354.63       D7    +4      M2
  10    2616.26       E7   -14      M3
  11    2877.88   F#/Gb7   -49   a4/d5
  12    3139.51       G7    +2      P5
  13    3401.13   G#/Ab7   +41      m6
  14    3662.76   A#/Bb7   -31      m7
  15    3924.38       B7   -12      M7
...


# Get a Random Modal Key and Search for a Backing Track

In [40]:
key = random.randint(0, len(note_names)-1)
mode = random.randint(0, len(modes)-1)

modal_key = note_names[key] + " " + modes[mode]
print(pyfiglet.figlet_format(modal_key,font='slant'))

url = "https://www.youtube.com/results?search_query=backing+track+" + urllib.parse.quote(note_names[key]) + "+" + modes[mode]

button = widgets.Button(description='Find a backing track')
button.on_click(lambda b: webbrowser.open(url))
display(button)

   ______  __ __     ___              ___           
  / ____/_/ // /_   /   | ___  ____  / (_)___ _____ 
 / / __/_  _  __/  / /| |/ _ \/ __ \/ / / __ `/ __ \
/ /_/ /_  _  __/  / ___ /  __/ /_/ / / / /_/ / / / /
\____/ /_//_/    /_/  |_\___/\____/_/_/\__,_/_/ /_/ 
                                                    



Button(description='Find a backing track', style=ButtonStyle())